# Advanced Macroeconomics 1: Homework 3

## Settings

In [ ]:
######################################################
# 📦 Package Imports
######################################################

# === Data Manipulation ===
import pandas as pd                # DataFrame handling
import numpy as np                # Numerical operations
import random                     # For reproducibility

# === Visualization ===
import matplotlib.pyplot as plt   # General plotting
import seaborn as sns             # Statistical visualization

# === Time Series & Statistical Models ===
import statsmodels.api as sm
from statsmodels.tsa.stattools import (
    adfuller,                     # Augmented Dickey-Fuller Test
    kpss,                         # KPSS Test
    acf, pacf                     # Autocorrelation functions
)
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import (
    plot_acf,                     # ACF plot
    plot_pacf                     # PACF plot
)
from statsmodels.stats.diagnostic import acorr_ljungbox  # Ljung-Box test
from statsmodels.tsa.api import VAR                      # VAR model
from arch.unitroot import PhillipsPerron, ZivotAndrews   # Unit root tests

# === Machine Learning Metrics ===
from sklearn.metrics import mean_squared_error           # Model evaluation

# === Optional External Data Sources ===
import ipeadatapy as ipea                                # IPEA database (optional)

# === Utilities ===
from collections import Counter
import warnings
warnings.filterwarnings("ignore")

# === Reproducibility ===
seed = 42
np.random.seed(seed)
random.seed(seed)

# === Custom Color ===
red_color = (162 / 255, 37 / 255, 56 / 255)




In [2]:
warnings.filterwarnings("ignore")

## Questions

### Question 1

Obtain the monthly time series for the following variables: IPCA inflation, IBC-Br (series 24363 from the Brazilian Central Bank), SELIC interest rate, commodity price index (series 27574 from the Central Bank), and the exchange rate. Construct a measure of the real interest rate using the IPCA as the deflator. Use data starting from January 2000.

With these series, estimate a Vector Autoregression (VAR) model using monthly data, imposing a recursive identification scheme (Cholesky decomposition). The ordering of the variables in the VAR should be as follows: IPCA inflation, real interest rate (constructed), commodity price index, IBC-Br, and exchange rate.

#### Data

In [7]:
# Here we use a keyword to search for available time series or browse them manually on the IPEA website
ipea.metadata()

,CODE,NAME,COMMENT,LAST UPDATE,BIG THEME,SOURCE ACRONYM,SOURCE,SOURCE URL,FREQUENCY,MEASURE,UNIT,SERIES STATUS,THEME CODE,COUNTRY,NUMERICA
0,ABATE_ABPEAV,Abate - aves - peso das carcaças,O abate de animais é mensurado por sua quantid...,2025-03-18T13:00:00.673-03:00,Macroeconômico,IBGE/Coagro,Instituto Brasileiro de Geografia e Estatístic...,www.ibge.gov.br,Anual,Tonelada,mil,A,1,BRA,True
1,ABATE_ABQUBV,Abate - bovinos - quantidade,O abate de animais é mensurado por sua quantid...,2025-03-18T13:00:00.673-03:00,Macroeconômico,IBGE/Coagro,Instituto Brasileiro de Geografia e Estatístic...,www.ibge.gov.br,Anual,Cabeça,mil,A,1,BRA,True
2,ABATE12_ABPEVA12,Abate - vacas - peso das carcaças,O abate de animais é mensurado por sua quantid...,2025-03-18T12:26:00.49-03:00,Macroeconômico,IBGE/Coagro,Instituto Brasileiro de Geografia e Estatístic...,www.ibge.gov.br,Mensal,Tonelada,mil,I,1,BRA,True
3,ABATE12_ABQUBO12,Abate - bois - quantidade,O abate de animais é mensurado por sua quantid...,2025-03-18T12:26:00.49-03:00,Macroeconômico,IBGE/Coagro,Instituto Brasileiro de Geografia e Estatístic...,www.ibge.gov.br,Mensal,Cabeça,mil,A,1,BRA,True
4,ABATE12_ABQUBV12,Abate - bovinos - quantidade,O abate de animais é mensurado por sua quantid...,2025-03-18T12:26:00.49-03:00,Macroeconômico,IBGE/Coagro,Instituto Brasileiro de Geografia e Estatístic...,www.ibge.gov.br,Mensal,Cabeça,mil,A,1,BRA,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2821,PNADCT_TXPARTCUF_SI,Taxa de participação - sem instrução ou equiva...,"Taxa de participação na força de trabalho, na ...",2025-02-18T12:24:51.957-03:00,Social,IBGE/PNAD Contínua,Instituto Brasileiro de Geografia e Estatístic...,http://www.ibge.gov.br/home/estatistica/indica...,Trimestral,(%),None,None,110,None,True
2822,PNAD_IAGRV,Domicílios com insegurança alimentar grave,Distribuição percentual dos domicílios de acor...,2024-06-20T10:44:01.677-03:00,Social,IBGE/PNAD Contínua,Instituto Brasileiro de Geografia e Estatístic...,http://www.ibge.gov.br/home/estatistica/indica...,Decenal,(%),None,None,111,None,True
2823,PNAD_IALEV,Domicílios com insegurança alimentar leve,Distribuição percentual dos domicílios de acor...,2024-06-20T10:44:01.68-03:00,Social,IBGE/PNAD Contínua,Instituto Brasileiro de Geografia e Estatístic...,http://www.ibge.gov.br/home/estatistica/indica...,Decenal,(%),None,None,111,None,True
2824,PNAD_IAMOD,Domicílios com insegurança alimentar moderada,Distribuição percentual dos domicílios de acor...,2024-06-20T10:44:01.683-03:00,Social,IBGE/PNAD Contínua,Instituto Brasileiro de Geografia e Estatístic...,http://www.ibge.gov.br/home/estatistica/indica...,Decenal,(%),None,None,111,None,True


In [8]:
# IPCA series

# Downloading the IPCA series from IPEA
df_ipca = ipea.timeseries('PRECOS12_IPCA12')

# Renaming columns for clarity
df_ipca = df_ipca.rename(columns={
    'RAW DATE': 'date',
    'VALUE (-)': 'IPCA',
    'CODE': 'code',
    'DAY': 'day',
    'MONTH': 'month',
    'YEAR': 'year'
})

# Drop unnecessary columns
df_ipca = df_ipca.drop(columns=['code', 'day', 'month', 'year'], errors='ignore')

# Convert 'date' column to datetime (with timezone awareness handling)
df_ipca['date'] = pd.to_datetime(df_ipca['date'], errors='coerce', utc=True)

# Set 'date' as index
df_ipca.set_index('date', inplace=True)

# Remove timezone to make index tz-naive (match other DataFrames)
df_ipca.index = df_ipca.index.tz_convert(None)
df_ipca.index = df_ipca.index.to_period('M').to_timestamp()

# Sort by date
df_ipca = df_ipca.sort_index()

# Calculate monthly inflation from IPCA index
df_ipca['monthly_inflation'] = df_ipca['IPCA'] / df_ipca['IPCA'].shift(1) - 1

df_ipca['monthly_inflation'] = df_ipca['monthly_inflation'].round(5)
df_ipca['IPCA'] = df_ipca['IPCA'].round(5)

# Preview
df_ipca.head()


,IPCA,monthly_inflation
date,,
1979-12-01,0.0,NaN
1980-01-01,0.0,0.06616
1980-02-01,0.0,0.04617
1980-03-01,0.0,0.06038
1980-04-01,0.0,0.05286


In [9]:
# IBC-Br series

# Defining the series code and the date range
series_code = 24363
start_date = '01/01/2000'
end_date = '31/12/2025'

# Building the API URL
url = f'https://api.bcb.gov.br/dados/serie/bcdata.sgs.{series_code}/dados?formato=json&dataInicial={start_date}&dataFinal={end_date}'

# Reading the JSON data directly from the API
df_ibcbr = pd.read_json(url)

# Converting the 'date' column to datetime format
df_ibcbr['date'] = pd.to_datetime(df_ibcbr['data'], format='%d/%m/%Y')

# Converting the 'valor' column to numeric type
df_ibcbr['valor'] = pd.to_numeric(df_ibcbr['valor'], errors='coerce')

# Renaming columns for clarity
df_ibcbr = df_ibcbr.rename(columns={
    'valor': 'IBC-Br',
})

# Drop columns
df_ibcbr = df_ibcbr.drop(columns=['data'], errors='ignore')

# Setting 'date' as the index
df_ibcbr.set_index('date', inplace=True)
df_ibcbr.index = df_ibcbr.index.to_period('M').to_timestamp()

# Calculate monthly variation
df_ibcbr['delta_IBCBR'] = df_ibcbr['IBC-Br'] / df_ibcbr['IBC-Br'].shift(1) - 1

df_ibcbr['IBC-Br'] = df_ibcbr['IBC-Br'].round(5)
df_ibcbr['delta_IBCBR'] = df_ibcbr['delta_IBCBR'].round(5)

# Preview
df_ibcbr.head()

,IBC-Br,delta_IBCBR
date,,
2003-01-01,67.44572,NaN
2003-02-01,69.20861,0.02614
2003-03-01,72.53661,0.04809
2003-04-01,71.67673,-0.01185
2003-05-01,70.35122,-0.01849


In [10]:
# SELIC Acum series

# Downloading the SELIC series from IPEA
df_selic = ipea.timeseries('BM12_TJOVER12')

# Renaming columns for clarity
df_selic = df_selic.rename(columns={
    'RAW DATE': 'date',
    'VALUE ((% a.m.))': 'SELIC',
    'CODE': 'code',
    'DAY': 'day',
    'MONTH': 'month',
    'YEAR': 'year'
})

# Dropping unnecessary columns
df_selic = df_selic.drop(columns=['code', 'day', 'month', 'year'], errors='ignore')

# Converting 'date' column to datetime (handling timezone if needed)
df_selic['date'] = pd.to_datetime(df_selic['date'], errors='coerce', utc=True)

# Setting 'date' as the index
df_selic.set_index('date', inplace=True)

# Removing timezone to make the index tz-naive
df_selic.index = df_selic.index.tz_convert(None)

# Converting index to monthly frequency (YYYY-MM-01)
df_selic.index = df_selic.index.to_period('M').to_timestamp()

# Sorting the DataFrame by date
df_selic = df_selic.sort_index()

# Convert SELIC to decimal
df_selic['selic_dec'] = df_selic['SELIC'] / 100

df_selic['SELIC'] = df_selic['SELIC'].round(5)
df_selic['selic_dec'] = df_selic['selic_dec'].round(5)

# Preview
df_selic.head()


,SELIC,selic_dec
date,,
1974-01-01,1.46,0.0146
1974-02-01,1.15,0.0115
1974-03-01,1.16,0.0116
1974-04-01,1.21,0.0121
1974-05-01,1.24,0.0124


In [11]:
# Commodity price index series

commodity_code = 27574
start_date = '01/01/2000'
end_date = '31/12/2025'

# Building the API URL
url_commodity = f'https://api.bcb.gov.br/dados/serie/bcdata.sgs.{commodity_code}/dados?formato=json&dataInicial={start_date}&dataFinal={end_date}'

# Reading the data from the API
df_commodity = pd.read_json(url_commodity)

# Converting 'data' to datetime and 'valor' to numeric
df_commodity['data'] = pd.to_datetime(df_commodity['data'], format='%d/%m/%Y', utc=True)
df_commodity['valor'] = pd.to_numeric(df_commodity['valor'], errors='coerce')

# Renaming columns for clarity
df_commodity = df_commodity.rename(columns={'data': 'date', 'valor': 'Commodity_Price_Index'})

# Setting date as index
df_commodity.set_index('date', inplace=True)

# Removing timezone to make index tz-naive
df_commodity.index = df_commodity.index.tz_convert(None)

# Converting index to monthly frequency (YYYY-MM-01)
df_commodity.index = df_commodity.index.to_period('M').to_timestamp()

# Sorting by date
df_commodity = df_commodity.sort_index()


# Calculate monthly inflation from IPCA index
df_commodity['monthly_Commodity_inflation'] = df_commodity['Commodity_Price_Index'] / df_commodity['Commodity_Price_Index'].shift(1) - 1

df_commodity['Commodity_Price_Index'] = df_commodity['Commodity_Price_Index'].round(5)
df_commodity['monthly_Commodity_inflation'] = df_commodity['monthly_Commodity_inflation'].round(5)

# Preview
df_commodity.head()


,Commodity_Price_Index,monthly_Commodity_inflation
date,,
2000-01-01,51.41,NaN
2000-02-01,50.22,-0.02315
2000-03-01,49.51,-0.01414
2000-04-01,50.31,0.01616
2000-05-01,53.18,0.05705


In [12]:
# Exchange rate series code (Dollar - commercial rate, selling)

# Downloading the series from IPEA
df_fx = ipea.timeseries('PAN12_ERV12')

# Renaming columns for clarity
df_fx = df_fx.rename(columns={
    'RAW DATE': 'date',
    'VALUE (R$)': 'FX_Rate',  # Corrigido! Estava escrito 'SELIC' por engano
    'CODE': 'code',
    'DAY': 'day',
    'MONTH': 'month',
    'YEAR': 'year'
})

# Dropping unnecessary columns
df_fx = df_fx.drop(columns=['code', 'day', 'month', 'year'], errors='ignore')

# Converting 'date' to datetime with timezone handling
df_fx['date'] = pd.to_datetime(df_fx['date'], errors='coerce', utc=True)

# Setting 'date' as index
df_fx.set_index('date', inplace=True)

# Removing timezone to make index tz-naive
df_fx.index = df_fx.index.tz_convert(None)

# Converting index to monthly frequency (YYYY-MM-01)
df_fx.index = df_fx.index.to_period('M').to_timestamp()

# Sorting by date
df_fx = df_fx.sort_index()

# Calculate monthly inflation from IPCA index
df_fx['delta_FX_Rate'] = df_fx['FX_Rate'] / df_fx['FX_Rate'].shift(1) - 1

df_fx['FX_Rate'] = df_fx['FX_Rate'].round(5)
df_fx['delta_FX_Rate']  = df_fx['delta_FX_Rate'] .round(5)


# Preview
df_fx.head()

,FX_Rate,delta_FX_Rate
date,,
1930-01-01,0.0,NaN
1930-02-01,0.0,0.00000
1930-03-01,0.0,-0.03333
1930-04-01,0.0,-0.02299
1930-05-01,0.0,0.00000


In [13]:
df = df_ibcbr.join([df_ipca, df_selic, df_commodity, df_fx], how='inner')

In [14]:
# Calculate real interest rate using exact Fisher formula
df['real_interest_rate'] = (1 + df['selic_dec']) / (1 + df['monthly_inflation']) - 1

df['real_interest_rate']  = df['real_interest_rate'].round(5)

In [15]:
# Select the variables of interest for the VAR
vars_var = df[['monthly_inflation', 'real_interest_rate', 'Commodity_Price_Index', 'IBC-Br', 'FX_Rate']]

# Check how many missing values there are in each column
print(vars_var.isna().sum())

monthly_inflation        0
real_interest_rate       0
Commodity_Price_Index    0
IBC-Br                   0
FX_Rate                  0
dtype: int64


#### Plot series

In [16]:
# Graph
window_size = 12

# Loop through each variable in the VAR dataset
for column in vars_var_clean.columns:
    
    # Calculate moving average and moving standard deviation
    moving_mean = vars_var_clean[column].rolling(window=window_size).mean()
    moving_std = vars_var_clean[column].rolling(window=window_size).std()
    
    # Calculate the overall mean and standard deviation of the series
    mean_value = vars_var_clean[column].mean()
    std_dev = vars_var_clean[column].std()
    
    # Create the plot
    plt.figure(figsize=(12, 3.5))
    
    # Plot the original series
    plt.plot(vars_var_clean.index, vars_var_clean[column], color=rgb_color, linewidth=1.5, label=f"{column}")
    
    # Plot the overall mean line
    plt.axhline(mean_value, color='black', linestyle='--', linewidth=1, label='Mean')
    
    # Plot the moving average
    plt.plot(vars_var_clean.index, moving_mean, color='black', linestyle='-', linewidth=1.5, label="Moving Average")
    
    # Plot the ±1 moving standard deviation bands
    plt.fill_between(vars_var_clean.index, 
                     moving_mean - moving_std, 
                     moving_mean + moving_std, 
                     color='grey', alpha=0.3, label='±1 Moving Std Dev')

    # Add title and customize plot
    plt.title(f"{column}", fontsize=14, fontweight='bold')
    plt.legend(fontsize=7, loc='best')
    plt.xlabel('')
    plt.ylabel('')
    plt.tight_layout()
    plt.show()

NameError: name 'vars_var_clean' is not defined

#### Choose lags for VAR model based on AIC, SBC, and HQ criteria



In [ ]:
# Fit VAR model
model = VAR(vars_var)

# Initialize dictionaries to store info criteria for each lag
aic_dict = {}
bic_dict = {}
hqic_dict = {}

# Test lags from 1 to 12
for lag in range(1, 13):
    result = model.fit(lag)
    aic_dict[lag] = result.aic
    bic_dict[lag] = result.bic
    hqic_dict[lag] = result.hqic

# Create DataFrame with all criteria
criteria_table = pd.DataFrame({
    'AIC': pd.Series(aic_dict),
    'BIC': pd.Series(bic_dict),
    'HQIC': pd.Series(hqic_dict)
})

print("Information Criteria by Lag:")
print(criteria_table)

# Find which lag each criterion selected
selected_orders = {
    'aic': criteria_table['AIC'].idxmin(),
    'bic': criteria_table['BIC'].idxmin(),
    'hqic': criteria_table['HQIC'].idxmin()
}

print("\nLag selected by each criterion:")
print(selected_orders)

# Count how many times each lag was selected
criteria_counts = Counter(selected_orders.values())
print("\nLag chosen by multiple criteria:")
print(criteria_counts)

# Choose the most common lag
most_common_lag = criteria_counts.most_common(1)[0][0]
print(f"\nFinal selected lag: {most_common_lag}")

# Fit final VAR model
results = model.fit(most_common_lag)


Information Criteria by Lag:
          AIC        BIC       HQIC
1  -24.112015 -23.706763 -23.949191
2  -24.576347 -23.831357 -24.276987
3  -24.827729 -23.741142 -24.391055
4  -24.943817 -23.513755 -24.369044
5  -24.985244 -23.209812 -24.271578
6  -25.117341 -22.994627 -24.263982
7  -25.377620 -22.905693 -24.383760
8  -25.427011 -22.603923 -24.291834
9  -25.471057 -22.294841 -24.193740
10 -25.544863 -22.013534 -24.124576
11 -25.801713 -21.913267 -24.237617
12 -25.993326 -21.745739 -24.284574

Lag selected by each criterion:
{'aic': np.int64(12), 'bic': np.int64(2), 'hqic': np.int64(3)}

Lag chosen by multiple criteria:
Counter({np.int64(12): 1, np.int64(2): 1, np.int64(3): 1})

Final selected lag: 12


#### Estimate the VAR model and perform the diagnostic/specification tests as discussed in class.


#### Data